## The Reutres dataset

A set of short newswires and their topics, published by Reuters in 1986. There are 46 different topics and each topic has at least 10 examples in the training set.

### Loading the Reuters dataset

In [17]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.utils import to_categorical # Converts a class vector (integers) to binary class matrix.
from tensorflow.keras import models
from tensorflow.keras import layers


import numpy as np
import pandas as pd

In [2]:
# Restrict the data to the 10,000 most frequently occurring words found in the data.
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

/home/ricardo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/datasets/reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/home/ricardo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/datasets/reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [3]:
print("Train shape:")
print(train_data.shape)
print(train_labels.shape)
print("\nTest shape:")
print(test_data.shape)
print(test_labels.shape)

Train shape:
(8982,)
(8982,)

Test shape:
(2246,)
(2246,)


In [4]:
print(train_data[0]) # word indices

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [5]:
print(train_labels[0]) # Topic index (integer between 0 and 45)

3


### Decoding newswires back to text

In [6]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]]) # offset by 3 cause reserved indices
print(decoded_newswire)

? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3


### Preparing the data

#### Encoding the data

In [7]:
def vectorize_seq(sequences, dim=10000):
    results = np.zeros((len(sequences), dim))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [8]:
x_train = vectorize_seq(train_data) # Vectorized train data
x_test = vectorize_seq(test_data) # Vectorized test data

In [9]:
# To vectorize the labels
one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

### Building the network

In [10]:
model = models.Sequential()

model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))

# For each input sample, the network will output a 46-dimensional vector.
model.add(layers.Dense(46, activation='softmax')) # Softmax converts a real vector to a vector of categorical probabilities.

### Compiling the model

In [11]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Validating

#### Setting aside a validation set

In [12]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

### Training the model

In [15]:
# For 20 epochs
EPOCHS = 20
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=EPOCHS,
                    batch_size=512,
                    validation_data=(x_val, y_val))

Epoch 1/20
16/16 [==============================] - 0s 14ms/step - loss: 0.0894 - accuracy: 0.9588 - val_loss: 1.3397 - val_accuracy: 0.7880
Epoch 2/20
16/16 [==============================] - 0s 11ms/step - loss: 0.0844 - accuracy: 0.9595 - val_loss: 1.3041 - val_accuracy: 0.7940
Epoch 3/20
16/16 [==============================] - 0s 11ms/step - loss: 0.0869 - accuracy: 0.9607 - val_loss: 1.3331 - val_accuracy: 0.7920
Epoch 4/20
16/16 [==============================] - 0s 11ms/step - loss: 0.0880 - accuracy: 0.9588 - val_loss: 1.4106 - val_accuracy: 0.7860
Epoch 5/20
16/16 [==============================] - 0s 11ms/step - loss: 0.0882 - accuracy: 0.9582 - val_loss: 1.3486 - val_accuracy: 0.7880
Epoch 6/20
16/16 [==============================] - 0s 10ms/step - loss: 0.0856 - accuracy: 0.9602 - val_loss: 1.3301 - val_accuracy: 0.7900
Epoch 7/20
16/16 [==============================] - 0s 10ms/step - loss: 0.0863 - accuracy: 0.9603 - val_loss: 1.3951 - val_accuracy: 0.7860
Epoch 8/20
16

In [20]:
history_df = pd.DataFrame(history.history)
history_df

,loss,accuracy,val_loss,val_accuracy
0,0.089420,0.958782,1.339677,0.788
1,0.084371,0.959534,1.304138,0.794
2,0.086857,0.960661,1.333124,0.792
3,0.087997,0.958782,1.410606,0.786
4,0.088196,0.958156,1.348563,0.788
5,0.085586,0.960160,1.330065,0.790
6,0.086258,0.960286,1.395137,0.786
7,0.083834,0.958908,1.392447,0.779
8,0.083787,0.958657,1.362253,0.786
9,0.087706,0.957404,1.350576,0.784
